In [1]:
import sys, os
import numpy as np
import pandas as pd
import copy

sys.path.append("../../..")
from src import bounded_mean_1d

### Create raw dataframe

In [2]:
df = pd.read_csv("../../../data/par2_nebd_embryos.csv")
df_quantification = pd.read_csv("../../../data/par2_nebd_quantification.csv")
df_quantification_grouped = df_quantification.groupby("EmbryoID")

df_summary = pd.DataFrame()
for i, r in df_quantification_grouped:
    cyt = np.mean(r["Cytoplasmic signal"])
    mem_tot = np.mean(r["Membrane signal"])
    mem_ant = bounded_mean_1d(r["Membrane signal"], [0.4, 0.6])
    mem_post = bounded_mean_1d(r["Membrane signal"], [0.9, 0.1])
    p = {
        "EmbryoID": i,
        "Cyt": cyt,
        "Mem_tot": mem_tot,
        "Mem_ant": mem_ant,
        "Mem_post": mem_post,
    }
    df_summary = df_summary.append(pd.DataFrame(p, index=[i]))
df = pd.merge(df, df_summary, on="EmbryoID")

# Filter
df = df[df.Line.isin(["lp637", "nwg201", "nwg338", "nwg369"])]

# # Remove points below threshold
# mem_thresh = 4e2
# cyt_thresh = 2e2
# df = df[df.Mem_post > mem_thresh]
# df = df[df.Cyt > cyt_thresh]

# Add uniform vs polarised column
unipol = ["Pol" if x in ["lp637", "nwg338"] else "Uni" for x in df.Line]
df["UniPol"] = unipol

# Add genotype column
genotype = ["WT" if x in ["lp637", "nwg201"] else "L109R" for x in df.Line]
df["Genotype"] = genotype

### Create dataframes with absolute concentrations

In [3]:
def create_dataframe(D, c):
    # D = diameter of molecules (um)
    # c = cytoplasmic concentration in wt cells (M)
    _df = copy.deepcopy(df)
    mem_norm = np.mean(
        _df[(df.Line == "lp637") * (_df.RNAi == "wt")].Mem_post.to_numpy()
    )
    cyt_norm = np.mean(_df[(df.Line == "lp637") * (_df.RNAi == "wt")].Cyt.to_numpy())
    xc = c
    xm = (mem_norm / cyt_norm) * (1 / D) * c
    _df.Mem_tot *= xm / mem_norm
    _df.Mem_ant *= xm / mem_norm
    _df.Mem_post *= xm / mem_norm
    _df.Cyt *= xc / cyt_norm
    return _df

In [4]:
# Main figure: D=5nm, c=10.4nM
df_main = create_dataframe(0.005, 1.04e-8)
df_main.to_csv("df_main.csv")

# Supplement 1: D=0.5nm, c=10.4nM
df_s1 = create_dataframe(0.0005, 1.04e-8)
df_s1.to_csv("df_s1.csv")

# Supplement 2: D=50nm, c=10.4nM
df_s2 = create_dataframe(0.05, 1.04e-8)
df_s2.to_csv("df_s2.csv")

# # Supplement 3: D=5.451nm, c=4.1nM
# df_s3 = create_dataframe(0.005451, 4.1e-9)
# df_s3.to_csv('df_s3.csv')

# # Supplement 4: D=5.451nm, c=410nM
# df_s4 = create_dataframe(0.005451, 4.1e-7)
# df_s4.to_csv('df_s4.csv')